In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import peteblast as pb

In [28]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

from elasticsearch.helpers import bulk

In [120]:
from Bio import SeqIO
import itertools as it

with open('data/bacteria.nr.100.faa', 'r') as f:
    parser = SeqIO.parse(f, 'fasta')
    records = list(it.islice(parser, 0, 10000000))

In [121]:
from skbio.alignment import StripedSmithWaterman


In [122]:
%%time

import hashlib
import peteblast as pb


def index_records(records):
    for i,rec in enumerate(records):
        if i % 10000 == 0:
            print(i)
        seq = str(rec.seq)
        seq_hash = hashlib.sha256(seq.encode('utf8')).hexdigest()
        minimizers = pb.minimizers(seq, k=5, w=10)
                
        seq_meta = {
                    "_index": "seqs",
                    "id": rec.id,
                    "name": rec.name,
                    "description": rec.description,
                    "seq": str(rec.seq),
                    "mins": list(minimizers)
                   }
        yield seq_meta
        
bulk(es, index_records(records))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000


Traceback (most recent call last):
  File "/Users/pete/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pete/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pete/miniconda3/lib/python3.8/http/client.py", line 1332, in getresponse
    response.begin()
  File "/Users/pete/miniconda3/lib/python3.8/http/client.py", line 303, in begin
    version, status, reason = self._read_status()
  File "/Users/pete/miniconda3/lib/python3.8/http/client.py", line 264, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pete/miniconda3/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [123]:
import json

In [124]:
%%time

def minimizer_counts(minimizers):
    out = []
    for m in minimizers:
        out += ["{}", json.dumps({"size": 0, "query": {"match": {"mins": m}}})]

    res = es.msearch(index='seqs', body=out)
    results = dict([(m, r['hits']['total']['value']) for m,r in zip(minimizers, res['responses'])])
    
    return results

ret = minimizer_counts(['AIGNE', 'AIGXE'])
ret

CPU times: user 1.85 ms, sys: 9.98 ms, total: 11.8 ms
Wall time: 206 ms


{'AIGNE': 144, 'AIGXE': 0}

In [138]:
%%time

def get_seqs(minimizer):
    res = es.search(index='seqs', body={"size": 1000, "_source": ["id"], "query": {"match": {"mins": minimizer}}})
    
#     print('res:', res)
    return [r['_id'] for r in res['hits']['hits']]

len(get_seqs('AIGNE'))

CPU times: user 1.98 ms, sys: 1.54 ms, total: 3.52 ms
Wall time: 31.4 ms


144

In [142]:
%%time

import collections as col

def search(query, results_frac=0.5):
    mins = [m for m in pb.minimizers(query, 5, 10)]
    res = minimizer_counts(mins)
    sorted_res = sorted(
        res.items(),
        key=lambda x: x[1],
    )

    seqs_found = col.defaultdict(int)
    MIN_TO_CHECK = 10
    num_to_check = max(int(len(mins) * results_frac), MIN_TO_CHECK)
    for _, (minimizer,count) in enumerate(sorted_res[:num_to_check]):
        seqs = get_seqs(minimizer)
        
        for seq in seqs:
            seqs_found[seq] += 1
    return sorted(seqs_found.items(), key=lambda x: -x[1])

search('MAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRGGAHDVYDTSAFTVPGHYTYRVRGVTSAGTPISDWSEMGA')

CPU times: user 10.3 ms, sys: 2.11 ms, total: 12.5 ms
Wall time: 134 ms


[('cH9QKHQB2kL29O8aiZB_', 8),
 ('cX9QKHQB2kL29O8aiZB_', 8),
 ('WH9QKHQB2kL29O8a4pRH', 8),
 ('WX9QKHQB2kL29O8a4pRH', 8),
 ('QH9RKHQB2kL29O8aHJgc', 8),
 ('QX9RKHQB2kL29O8aHJgc', 8),
 ('aX9SKHQB2kL29O8aM7-I', 8),
 ('an9SKHQB2kL29O8aM7-I', 8),
 ('V4FvKHQB2kL29O8aTUe7', 8),
 ('WIFvKHQB2kL29O8aTUe7', 8),
 ('yH9QKHQB2kL29O8aiZCB', 3),
 ('sH9QKHQB2kL29O8a4pRI', 3),
 ('mH9RKHQB2kL29O8aHJge', 3),
 ('wX9SKHQB2kL29O8aM7-J', 3),
 ('r4FvKHQB2kL29O8aTUe9', 3),
 ('xX9QKHQB2kL29O8aiZCB', 2),
 ('GX9QKHQB2kL29O8ajpLC', 2),
 ('rX9QKHQB2kL29O8a4pRI', 2),
 ('AX9QKHQB2kL29O8a5JbL', 2),
 ('lX9RKHQB2kL29O8aHJge', 2),
 ('6X9RKHQB2kL29O8aHplq', 2),
 ('vn9SKHQB2kL29O8aM7-J', 2),
 ('En9SKHQB2kL29O8aNcGk', 2),
 ('rIFvKHQB2kL29O8aTUe9', 2),
 ('AIFvKHQB2kL29O8aUUnP', 2),
 ('oId1KHQB2kL29O8afCmn', 1),
 ('vYd2KHQB2kL29O8aQPZ4', 1),
 ('T4h2KHQB2kL29O8atm7r', 1),
 ('RIFTKHQB2kL29O8asEO7', 1),
 ('MoJwKHQB2kL29O8a1MsS', 1),
 ('-Yd2KHQB2kL29O8aBq_m', 1),
 ('Joh2KHQB2kL29O8ajjjv', 1),
 ('n4V0KHQB2kL29O8aE_Gm', 1),
 ('gYNyKHQ

In [134]:
from skbio.alignment import StripedSmithWaterman
import time

In [128]:
def get_sequence(_id):
    return es.get(index='seqs', id=_id)['_source']


In [129]:
subst_mat = pb.calc_subst_mat()

In [137]:
%%time

def results(query, max_results=30):    
    print("query:", query)
    t1 = time.time()
    id_counts = search(query)
    t2 = time.time()
    
    print("time:", t2 - t1)
    ssw = StripedSmithWaterman(query, protein=True, substitution_matrix=subst_mat)
    results_out = []
    for _id, count in id_counts[:max_results]:
        seq = get_sequence(_id)
        desc = seq['description']
        sequence = str(seq['seq'])
        alignment = ssw(sequence)
        match = 0
        total = 0
        for q, t in zip(alignment.aligned_query_sequence, alignment.aligned_target_sequence):
            if q == "-" or t == "-":
                continue
            if q == t:
                match += 1
            total += 1
        identity = match / total
        results_out += [
            {
                "description": seq['description'],
                "aligned_query": alignment.aligned_query_sequence,
                "aligned_target": alignment.aligned_target_sequence,
                "target_seq": str(seq['seq']),
                "target_description": str(seq['description']),
                "score": int(alignment.optimal_alignment_score),
                "minimizer_matches": count,
                "identity": "{:.2f}".format(identity),
            }
        ]
    return results_out

results('MAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRGGAHDVYDTSAFTVPGHYTYRVRGVTSAGTPISDWSEMGA')

query: MAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRGGAHDVYDTSAFTVPGHYTYRVRGVTSAGTPISDWSEMGA
time: 0.716468095779419
CPU times: user 40.3 ms, sys: 11.7 ms, total: 52 ms
Wall time: 888 ms


[{'description': 'WP_006694969.1 lipase [Selenomonas noxia]',
  'aligned_query': 'MAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRGGAHDVYDTSAFTVPGHYTYRVRGVTSAGTPISDWSEMGA',
  'aligned_target': 'MAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRGGAHDVYDTSAFTVPGHYTYRVRGVTSAGTPISDWSEMGA',
  'target_seq': 'MQVRMFGRSVGALLAVCVLFAALPASAAVRQVDRGTAQMGAAARKAQNIKAQTERERPENARASSAMLHWKAYPGAVRYAVRILRGTPGGTMTTVVSAERVYTTGVHLPLGAHGADEGLYWSVQPLDYRGTPLAQPSKPEALPRLAEDPTAPVLTAEFGEMAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRGGAHDVYDTSAFTVPGHYTYRVRGVTSAGTPISDWSEMGAFDVIGHTPIAAIGDSITHGGGAISVPPSYLLYDWETYCTVPVKNLGRSGDTTADMLARFEHDVLPFAPRVLIIMGGVNDYRSGVYGAESVRHLAALRDKCRAYGITPVFLTATPIRPALMAARMDIMTPPADWWTHRDYINNWVMQQEYSVDVSTVLSDADGELEAEYTTDGLHPDLVGKKYIGQTVDAYLRTHFAYAAGEAERRVQSFRAAGK',
  'target_description': 'WP_006694969.1 lipase [Selenomonas noxia]',
  'score': 436,
  'minimizer_matches': 8,
  'identity': '1.00'},
 {'description': 'WP_006696021.1 lipase [Selenomonas noxia]',
  'aligned_query': 'MAYAPLYPAYSWIPLSGQMHHEVEVYRREGGRSILLHTLRG